In [2]:
from pathlib import Path
from sklearn.model_selection import train_test_split
import os
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm
import shutil
import pandas as pd

In [3]:
class FramesToNpyConverter:
    def __init__(self, seq_len=37, max_hands=2):
        self.seq_len = seq_len
        self.max_hands = max_hands
        self.landmark_dim = 21 * 3 * max_hands

        mp_hands = mp.solutions.hands
        self.hands = mp_hands.Hands(
            static_image_mode=True,
            max_num_hands=max_hands,
            min_detection_confidence=0.6
        )

    def extract_landmarks(self, image):
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = self.hands.process(image_rgb)

        if not results.multi_hand_landmarks:
            return np.zeros(self.landmark_dim, dtype=np.float32)

        coords = []

        for hand in results.multi_hand_landmarks[: self.max_hands]:
            for lm in hand.landmark:
                coords.extend([lm.x, lm.y, lm.z])

        # pad missing hands
        missing = self.max_hands - len(results.multi_hand_landmarks)
        if missing > 0:
            coords.extend([0.0] * missing * 21 * 3)

        return np.array(coords, dtype=np.float32)

    def frames_folder_to_npy(self, frames_dir):
        frame_files = sorted(
            f for f in os.listdir(frames_dir)
            if f.lower().endswith((".jpg", ".png"))
        )

        sequence = []

        for f in frame_files:
            img = cv2.imread(os.path.join(frames_dir, f))
            if img is None:
                continue

            sequence.append(self.extract_landmarks(img))

        # handle rare edge cases
        if len(sequence) == 0:
            return np.zeros((self.seq_len, self.landmark_dim), dtype=np.float32)

        if len(sequence) != self.seq_len:
            idx = np.linspace(0, len(sequence) - 1, self.seq_len).astype(int)
            sequence = [sequence[i] for i in idx]

        return np.stack(sequence)

    def save(self, frames_dir, output_dir):
        os.makedirs(output_dir, exist_ok=True)

        seq = self.frames_folder_to_npy(frames_dir)

        video_name = os.path.basename(frames_dir.rstrip("/"))
        out_path = os.path.join(output_dir, video_name + ".npy")

        np.save(out_path, seq)
        return out_path


In [ ]:
FRAMES_ROOT = Path(r"data\\SL_frames_output")
NPY_OUTPUT  = Path(r"data\\SL_12_classes_npy")  
NPY_OUTPUT.mkdir(parents=True, exist_ok=True)

converter = FramesToNpyConverter(seq_len=37, max_hands=2)

# collect all video folders: class/video_id
video_dirs = []
for class_dir in FRAMES_ROOT.iterdir():
    if not class_dir.is_dir():
        continue
    for vid_dir in class_dir.iterdir():
        if vid_dir.is_dir():
            video_dirs.append(vid_dir)

print(f"Found {len(video_dirs)} video folders under {FRAMES_ROOT}")

errors = []
for vid_dir in tqdm(video_dirs, desc="Converting"):
    class_name = vid_dir.parent.name
    out_dir = NPY_OUTPUT / class_name
    out_dir.mkdir(parents=True, exist_ok=True)

    try:
        converter.save(str(vid_dir), str(out_dir))
    except Exception as e:
        errors.append((str(vid_dir), str(e)))

print(f"Done. Errors: {len(errors)}")
if errors:
    print("First few errors:")
    for p, msg in errors[:10]:
        print(" -", p, "->", msg)

Found 168 video folders under data\SL_frames_output


Converting: 100%|██████████| 168/168 [03:48<00:00,  1.36s/it]

Done. Errors: 0


In [5]:
ROOT = Path(r"data\SL_12_classes_npy")  # folder that currently has subfolders

# Move all npy files from subfolders into ROOT
for subdir in [p for p in ROOT.iterdir() if p.is_dir()]:
    class_name = subdir.name
    for npy_path in subdir.rglob("*.npy"):
        # rename to avoid collisions: e.g., bed_05629.npy
        new_name = f"{class_name}_{npy_path.name}"
        dest = ROOT / new_name

        # if you prefer overwrite, replace this check with: dest.unlink(missing_ok=True)
        if dest.exists():
            raise FileExistsError(f"Collision: {dest} already exists")

        shutil.move(str(npy_path), str(dest))

# Remove empty subfolders (deepest first)
for d in sorted([p for p in ROOT.rglob("*") if p.is_dir()], key=lambda x: len(x.parts), reverse=True):
    try:
        d.rmdir()  # only removes if empty
    except OSError:
        pass

print("Done flattening:", ROOT)

Done flattening: data\SL_12_classes_npy


In [ ]:
FRAMES_ROOT = "data\\jester_2_classes"
NPY_OUTPUT  = "data\\jester_2_classes_npy"

converter = FramesToNpyConverter(seq_len=37, max_hands=2)
os.makedirs(NPY_OUTPUT, exist_ok=True)

for video_folder in tqdm(os.listdir(FRAMES_ROOT)):
    frames_dir = os.path.join(FRAMES_ROOT, video_folder)
    if not os.path.isdir(frames_dir):
        continue

    try:
        converter.save(frames_dir, NPY_OUTPUT)
    except Exception as e:
        print("ERROR:", frames_dir, e)


100%|██████████| 6218/6218 [1:28:44<00:00,  1.17it/s]


In [6]:
SL_NPY = "data\\SL_12_classes_npy"
JESTER_NPY = "data\\jester_NO_npy"

SPLITS_DIR = "data\\splits"
os.makedirs(SPLITS_DIR, exist_ok=True)

TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

def collect_files_with_labels(root_dir, class_name, label):
    """Collect all .npy files with their class and label."""
    files = list(Path(root_dir).rglob("*.npy"))
    data = []
    for f in files:
        data.append({
            'filepath': str(f),
            'class': class_name,
            'label': label
        })
    return data

def create_split_csvs(yes_data, no_data, splits_dir):
    """Create train/val/test CSV files."""
    
    # Combine and create DataFrame
    all_data = yes_data + no_data
    df = pd.DataFrame(all_data)
    
    print(f"Total samples: {len(df)}")
    print(f"YES class: {len(yes_data)} files")
    print(f"NO class: {len(no_data)} files")
    
    # Split by class for stratification
    yes_df = df[df['class'] == 'yes']
    no_df = df[df['class'] == 'no']
    
    # Split YES class
    yes_train, yes_temp = train_test_split(
        yes_df, 
        test_size=0.2,
        random_state=42
    )
    yes_val, yes_test = train_test_split(
        yes_temp, 
        test_size=0.5,
        random_state=42
    )
    
    # Split NO class
    no_train, no_temp = train_test_split(
        no_df, 
        test_size=0.2,
        random_state=42
    )
    no_val, no_test = train_test_split(
        no_temp, 
        test_size=0.5,
        random_state=42
    )
    
    train_df = pd.concat([yes_train, no_train]).sample(frac=1, random_state=42).reset_index(drop=True)
    val_df = pd.concat([yes_val, no_val]).sample(frac=1, random_state=42).reset_index(drop=True)
    test_df = pd.concat([yes_test, no_test]).sample(frac=1, random_state=42).reset_index(drop=True)
    
    train_df.to_csv(os.path.join(splits_dir, 'train.csv'), index=False)
    val_df.to_csv(os.path.join(splits_dir, 'val.csv'), index=False)
    test_df.to_csv(os.path.join(splits_dir, 'test.csv'), index=False)
    
    # Print summary
    print("\n" + "="*70)
    print("Dataset Split Summary (80% Train / 10% Val / 10% Test):")
    print("="*70)
    
    for split_name, split_df in [('TRAIN', train_df), ('VAL', val_df), ('TEST', test_df)]:
        yes_count = len(split_df[split_df['class'] == 'yes'])
        no_count = len(split_df[split_df['class'] == 'no'])
        total = len(split_df)
        yes_pct = (yes_count / len(yes_df)) * 100
        no_pct = (no_count / len(no_df)) * 100
        print(f"{split_name:5s}: YES={yes_count:4d} ({yes_pct:5.1f}%), "
              f"NO={no_count:4d} ({no_pct:5.1f}%), Total={total:5d}")
    print("="*70)
    
    print(f"\nCSV files saved to: {splits_dir}")

# Collect files with labels
print("Collecting files...")
yes_data = collect_files_with_labels(SL_NPY, 'yes', 1)
no_data = collect_files_with_labels(JESTER_NPY, 'no', 0)

# Create split CSVs
create_split_csvs(yes_data, no_data, SPLITS_DIR)

Total samples: 6386
YES class: 168 files
NO class: 6218 files

Dataset Split Summary (80% Train / 10% Val / 10% Test):
TRAIN: YES= 134 ( 79.8%), NO=4974 ( 80.0%), Total= 5108
VAL  : YES=  17 ( 10.1%), NO= 622 ( 10.0%), Total=  639
TEST : YES=  17 ( 10.1%), NO= 622 ( 10.0%), Total=  639

CSV files saved to: data\splits
